In this notebook you will be investigating the fundamentals of reinforcement learning (RL). The first section describes the Markov decision process (MDP), which is a framework to help you design problems. The second section formulates an RL-driven solution for the MDP.



# Section 1. The Markov Decision Process (MDP)

- MDP is used to encapsulate the real-world
- MDP has 2 entities:
    1. **Agent**: RL algorithm
    2. **Environment**: real life or simulation

- MDP Interface: you need representations of:
    1. **State**: what to show to the agent. It's an observation of the environment, which contains everything outside of the agent
    2. **Action**: suggested by the agent
    3. **Reward**: you use it to fine tune the action choices
-  The environment’s step function accepts an **action**, then produces a new **state** and **reward**.

In [3]:
from collections import defaultdict, namedtuple
from enum import Enum
from typing import Tuple, List
import random
from IPython.display import clear_output

Point = namedtuple('Point', ['x', 'y'])
class Direction(Enum):
  NORTH = "⬆"
  EAST = "⮕"
  SOUTH = "⬇"
  WEST = "⬅"

  @classmethod
  def values(self):
    return [v for v in self]

class SimpleGridWorld(object):
    def __init__(self, width: int = 5, height: int = 5, debug: bool = False):
        self.width = width
        self.height = height
        self.debug = debug
        self.action_space = [d for d in Direction]
        self.reset()


    def reset(self):
        self.cur_pos = Point(x=0, y=(self.height - 1))
        self.goal = Point(x=(self.width - 1), y=0)
        # If debug, print state
        if self.debug:
            print(self)
        return self.cur_pos, 0, False

    # remember:  
    # The environment’s step function accepts an action, then produces a new state and reward.
    def step(self, action: Direction):
        # Depending on the action, mutate the environment state
        if action == Direction.NORTH:
            self.cur_pos = Point(self.cur_pos.x, self.cur_pos.y + 1)
        elif action == Direction.EAST:
            self.cur_pos = Point(self.cur_pos.x + 1, self.cur_pos.y)
        elif action == Direction.SOUTH:
            self.cur_pos = Point(self.cur_pos.x, self.cur_pos.y - 1)
        elif action == Direction.WEST:
            self.cur_pos = Point(self.cur_pos.x - 1, self.cur_pos.y)
        # Check if out of bounds
        if self.cur_pos.x >= self.width:
            self.cur_pos = Point(self.width - 1, self.cur_pos.y)
        if self.cur_pos.y >= self.height:
            self.cur_pos = Point(self.cur_pos.x, self.height - 1)
        if self.cur_pos.x < 0:
            self.cur_pos = Point(0, self.cur_pos.y)
        if self.cur_pos.y < 0:
            self.cur_pos = Point(self.cur_pos.x, 0)

        # If at goal, terminate
        is_terminal = self.cur_pos == self.goal

        # Constant -1 reward to promote speed-to-goal
        reward = -1

        # If debug, print state
        if self.debug:
            print(self)

        return self.cur_pos, reward, is_terminal
    
    
    # visualisation
    def __repr__(self):
        res = ""
        for y in reversed(range(self.height)):
            for x in range(self.width):
                if self.goal.x == x and self.goal.y == y:
                    if self.cur_pos.x == x and self.cur_pos.y == y:
                        res += "@"
                    else:
                        res += "o"
                    continue
                if self.cur_pos.x == x and self.cur_pos.y == y:
                    res += "x"
                else:
                    res += "_"
            res += "\n"
        return res

## Moving manually

In [6]:
s = SimpleGridWorld(debug=True)
print("☝ This shows a simple visualisation of the environment state.\n")
s.step(Direction.SOUTH)
print(s.step(Direction.SOUTH), "⬅ This displays the state and reward from the environment 𝐀𝐅𝐓𝐄𝐑 moving.\n")
s.step(Direction.SOUTH)
s.step(Direction.SOUTH)
s.step(Direction.EAST)
s.step(Direction.EAST)
s.step(Direction.EAST)
s.step(Direction.EAST)

x____
_____
_____
_____
____o

☝ This shows a simple visualisation of the environment state.

_____
x____
_____
_____
____o

_____
_____
x____
_____
____o

(Point(x=0, y=2), -1, False) ⬅ This displays the state and reward from the environment 𝐀𝐅𝐓𝐄𝐑 moving.

_____
_____
_____
x____
____o

_____
_____
_____
_____
x___o

_____
_____
_____
_____
_x__o

_____
_____
_____
_____
__x_o

_____
_____
_____
_____
___xo

_____
_____
_____
_____
____@



(Point(x=4, y=0), -1, True)

# Section 2. RL Solution to the MDP: Monte Carlo Methods

Monte Carlo methods are used in modern RL methods to randomly sample states and judge their value.

## Generating trajectories
The idea of MC techniques is that if we can sample the environment enough times, we can begin to build a picture of the output given any input.
If we capture enough tragectories (a trajectory is a one full pass through the env.), then we can see which states are advantagous.

We will implement a class that's capable of generating trajectories by passing in the environment as an `object`

In [116]:
class MonteCarloGeneration(object):
    def __init__(self, env: SimpleGridWorld, max_steps: int = 1000, debug: bool = False):
        self.env = env
        self.max_steps = max_steps
        self.debug = debug

    def run(self) -> List:
        buffer = []
        n_steps = 0
        state, _, _ = self.env.reset()
        terminal = False
        while not terminal:
            action = random.choice(self.env.action_space) # Random action. Try replacing this with Direction.EAST
            next_state, reward, terminal = self.env.step(action) # Take action in environment
            buffer.append((state, action, reward)) # Store the result
            state = next_state # Ready for the next step
            
            n_steps += 1
            if n_steps >= self.max_steps:
                if self.debug:
                    print("Terminated early due to large number of steps")
                terminal = True
        return buffer # return the log

In [117]:
env = SimpleGridWorld(debug=True) # Instantiate the environment
generator = MonteCarloGeneration(env=env, max_steps=10, debug=True)
trajectory = generator.run()
print([t[1].value for t in trajectory]) # Print chosen actions
print(f"total reward: {sum([t[2] for t in trajectory])}") # Print final reward


x____
_____
_____
_____
____o

x____
_____
_____
_____
____o

x____
_____
_____
_____
____o

x____
_____
_____
_____
____o

x____
_____
_____
_____
____o

_____
x____
_____
_____
____o

x____
_____
_____
_____
____o

x____
_____
_____
_____
____o

_x___
_____
_____
_____
____o

x____
_____
_____
_____
____o

x____
_____
_____
_____
____o

x____
_____
_____
_____
____o

Terminated early due to large number of steps
['⬅', '⬆', '⬆', '⬇', '⬆', '⬆', '⮕', '⬅', '⬆', '⬆']
total reward: -10


## Quantifying Value

We will quantofy the value isong *action-value function*: it is a measure of the value of taking a particular action, given all the experience (i.e. the previous trajectories)

First, let's make a class that runs an experiment by generating a full trajectory from start to termination

In [118]:
class MonteCarloExperiment(object):
    def __init__(self, generator: MonteCarloGeneration):
        self.generator = generator
        self.values = defaultdict(float)
        self.counts = defaultdict(float)


    def _to_key(self, state, action):
        return (state, action)

    def action_value(self, state, action) -> float:
        key = self._to_key(state, action)
        if self.counts[key] > 0:
            return self.values[key] / self.counts[key]
        else:
            return 0.0

    # find the value of an action, on average
    def run_episode(self) -> None:
        trajectory = self.generator.run() # Generate a trajectory
        episode_reward = 0
        for i, t in enumerate(reversed(trajectory)): # Starting from the terminal state
            state, action, reward = t
            key = self._to_key(state, action)
            episode_reward += reward  # Add the reward to the buffer
            self.values[key] += episode_reward # And add this to the value of this action
            self.counts[key] += 1 # Increment counter

In [128]:
env = SimpleGridWorld(debug=False) # Instantiate the environment - set the debug to true to see the actual movemen of the agent.
generator = MonteCarloGeneration(env=env, debug=True) # Instantiate the trajectory generator
agent = MonteCarloExperiment(generator=generator)
for i in range(4):
    agent.run_episode()
    # Point(3,0) is a step away from the goal @
    print(f"Run {i}: ", [agent.action_value(Point(3,0), d) for d in env.action_space])
    print([d.value for d in env.action_space])

Run 0:  [-20.0, -1.0, 0.0, -34.5]
['⬆', '⮕', '⬇', '⬅']
Run 1:  [-16.5, -1.0, 0.0, -34.666666666666664]
['⬆', '⮕', '⬇', '⬅']
Run 2:  [-77.4, -1.0, 0.0, -69.16666666666667]
['⬆', '⮕', '⬇', '⬅']
Run 3:  [-77.4, -1.0, 0.0, -69.16666666666667]
['⬆', '⮕', '⬇', '⬅']


In [120]:
def state_value_2d(env, agent):
    res = ""
    for y in reversed(range(env.height)):
      for x in range(env.width):
        if env.goal.x == x and env.goal.y == y:
          res += "   @  "
        else:
          state_value = sum([agent.action_value(Point(x,y), d) for d in env.action_space]) / len(env.action_space)
          res += f'{state_value:6.2f}'
        res += " | "
      res += "\n"
    return res

print(state_value_2d(env, agent))

-77.14 | -41.38 | -28.57 | -14.00 | -26.00 | 
-82.03 | -66.35 | -39.17 | -19.25 | -13.38 | 
-64.90 | -64.25 | -35.12 | -34.00 | -34.38 | 
-64.83 | -41.71 | -36.92 | -44.75 | -36.50 | 
-82.12 | -57.67 | -43.02 | -29.83 |    @   | 



note that the value increases the closer you get to the goal

## Generating Optimal Policies
The optimal policy is to choose the action that picks the highest expected return

In [148]:
env = SimpleGridWorld(debug=False) # Instantiate the environment - set the debug to true to see the actual movemen of the agent.
generator = MonteCarloGeneration(env=env, debug=True) # Instantiate the trajectory generator
agent = MonteCarloExperiment(generator=generator)
for i in range(1000):
    clear_output(wait=True)
    agent.run_episode()
    print(f"Iteration: {i}")
    print(state_value_2d(env, agent), flush=True)

Iteration: 999
-107.63 | -107.07 | -101.09 | -96.49 | -90.60 | 
-106.98 | -104.33 | -97.27 | -93.74 | -86.14 | 
-101.87 | -99.03 | -93.17 | -83.35 | -75.48 | 
-99.30 | -92.73 | -83.85 | -64.51 | -47.91 | 
-97.60 | -90.73 | -74.20 | -45.40 |    @   | 



In [149]:
def argmax(a):
    return max(range(len(a)), key=lambda x: a[x])

def next_best_value_2d(env, agent):
    res = ""
    for y in reversed(range(env.height)):
      for x in range(env.width):
        if env.goal.x == x and env.goal.y == y:
          res += "@"
        else:
          # Find the action that has the highest value
          loc = argmax([agent.action_value(Point(x,y), d) for d in env.action_space])
          res += f'{env.action_space[loc].value}'
        res += " | "
      res += "\n"
    return res

print(next_best_value_2d(env, agent))


⬇ | ⮕ | ⮕ | ⮕ | ⬇ | 
⮕ | ⮕ | ⬇ | ⬇ | ⬇ | 
⮕ | ⮕ | ⮕ | ⬇ | ⬇ | 
⮕ | ⮕ | ⮕ | ⬇ | ⬇ | 
⮕ | ⮕ | ⮕ | ⮕ | @ | 



By running it more times, we get better policy. Let's how policy converges

In [156]:
import time

env = SimpleGridWorld() # Instantiate the environment
generator = MonteCarloGeneration(env=env) # Instantiate the trajectory generator
agent = MonteCarloExperiment(generator=generator)
for i in range(1000):
    clear_output(wait=True)
    agent.run_episode()
    print(f"Iteration: {i}")
    print(state_value_2d(env, agent))
    print(next_best_value_2d(env, agent), flush=True)
    time.sleep(0.1)


Iteration: 999
-106.67 | -102.87 | -96.26 | -95.36 | -95.13 | 
-103.15 | -100.87 | -94.50 | -91.39 | -86.13 | 
-103.69 | -97.02 | -87.86 | -80.85 | -73.98 | 
-103.23 | -94.83 | -82.42 | -66.86 | -50.67 | 
-99.92 | -92.62 | -76.92 | -48.59 |    @   | 

⮕ | ⬇ | ⬆ | ⬆ | ⬇ | 
⬅ | ⮕ | ⬇ | ⬇ | ⬇ | 
⮕ | ⮕ | ⬇ | ⬇ | ⬇ | 
⮕ | ⮕ | ⮕ | ⬇ | ⬇ | 
⮕ | ⮕ | ⮕ | ⮕ | @ | 



Note that the agent is still totally random at this point. The agent started in the top left, so on average, it takes a lot of stumbling to find the goal. Therefore, for the points at the top left, furthest away from the goal, the agent will probably take many more random steps before it reachest the goal. That's why we get incorrect arrows at the top (especcially in the very first iterations)